Combining Tree

In [1]:
%%writefile Node.java
public class Node {
    enum CStatus{IDLE, FIRST, SECOND, RESULT, ROOT};
    boolean locked;
    CStatus cStatus;
    int firstValue, secondValue;
    int result;
    Node parent;
    public Node() {
        cStatus = CStatus.ROOT;
        locked = false;
    }
    public Node(Node myParent) {
        parent = myParent;
        cStatus = CStatus.IDLE;
        locked = false;
    }
    
    synchronized boolean precombine() throws InterruptedException{
        while (locked) wait();
        switch (cStatus) {
            case IDLE:
                cStatus = CStatus.FIRST;
                return true;
            case FIRST:
                locked = true;
                cStatus = CStatus.SECOND;
                return false;
            case ROOT:
                return false;
            default:
                throw new InterruptedException("unexpected Node state" + cStatus);
        }
    }
    
    synchronized int combine(int combined) throws InterruptedException{
        while (locked) wait();
        locked = true;
        firstValue = combined;
        switch (cStatus) {
            case FIRST:
                return firstValue;
            case SECOND:
                return firstValue + secondValue;
            default:
                throw new InterruptedException("unexpected Node state " + cStatus);
        }
    }
    
    synchronized int op(int combined)  throws InterruptedException{
        switch (cStatus) {
            case ROOT:
                int prior = result;
                result += combined;
                return prior;
            case SECOND:
                secondValue = combined;
                locked = false;
                notifyAll(); // wake up waiting threads
                while (cStatus != CStatus.RESULT) wait();
                locked = false;
                notifyAll();
                cStatus = CStatus.IDLE;
                return result;
            default:
                throw new InterruptedException("unexpected Node state");
        }
    }
    
    synchronized void distribute(int prior) throws InterruptedException{
        switch (cStatus) {
            case FIRST:
                cStatus = CStatus.IDLE;
                locked = false;
                break;
            case SECOND:
                result = prior + firstValue;
                cStatus = CStatus.RESULT;
                break;
            default:
                throw new InterruptedException("unexpected Node state");
        }
        notifyAll();
    }
}

Overwriting Node.java


In [2]:
%%writefile CombiningTree.java
import java.util.*;


public class CombiningTree {
    Node[] leaf;
    Node[] nodes;
    
    public CombiningTree(int width) {
        nodes = new Node[2*width -1];
        nodes[0] = new Node();
        for(int i=1; i<nodes.length; i++) {
            nodes[i] = new Node(nodes[(i-1)/2]);
        }
        leaf = new Node[width];
        for(int i=0; i<leaf.length; i++) {
            leaf[i] = nodes[nodes.length-i-1];
        }  
    }
    
    public int getAndIncrement() {
        Stack<Node> stack = new Stack<Node>();
        int ThreadID = (int) Thread.currentThread().getId();
        Node myLeaf = leaf[ThreadID % leaf.length];
        Node node = myLeaf;
        // precombining phase
        try{
            while (node.precombine()) {
                node = node.parent;
            }
        }catch(Exception e) {}
        Node stop = node;
        // combining phase
        int combined = 1;
        for (node = myLeaf; node != stop; node = node.parent) {
            try{combined = node.combine(combined);} catch (Exception e){}
            stack.push(node);
        }
        try{
            // operation phase
            int prior = stop.op(combined);
            // distribution phase
            while (!stack.empty()) {
                node = stack.pop();
                node.distribute(prior);
            }
            return prior;
        }catch(Exception e){}
        return -1;
        
    }
    
    public int get() {
        return nodes[0].result;
    }
}

Overwriting CombiningTree.java


In [15]:
%%writefile TestCombiningTree.java
import java.util.*;
import java.io.FileWriter;
import java.io.IOException;
import java.io.BufferedWriter;

class TestCombiningTree {
    static CombiningTree tree;
    //static int TH = 12, NUM = 100;
    static int TH, NUM;
    static int width;
    // tree: combining tree where threads do increment ops
    // TH: number of threads
    // NUM: number of increment operations each thread performs

    static Thread thread(int id) {
        return new Thread(() -> {
            
            long start = System.currentTimeMillis();
            for (int i=0; i<NUM; i++) {
               tree.getAndIncrement();
            }
            long stop = System.currentTimeMillis();
            log(id()+": done in "+(stop-start)+"ms");
        });
    }

    // Setup the combining tree for threads.
    static void setupTree() {
        width = (int) Math.ceil(TH/2);
        tree = new CombiningTree(width);
    }


    // Start threads doing increments using tree.
    static Thread[] startOps() {
        Thread[] t = new Thread[TH];
        for (int i=0; i<TH; i++)
          t[i] = thread(i);
        for (int i=0; i<TH; i++)
          t[i].start();
        return t;
    }

    // Wait until all threads done with increments.
    static void awaitOps(Thread[] t) {
        try {
        for (int i=0; i<TH; i++)
          t[i].join();
        } catch (InterruptedException e) {}
    }

    public static void main(String[] args) {
        TH = Integer.parseInt(args[0]);
        NUM = Integer.parseInt(args[1]);
        setupTree();
        log( width +"-leaves Combining tree.");
        log("Starting "+ TH +" threads doing increments ...");
        long start = System.currentTimeMillis();
        Thread[] t = startOps();
        awaitOps(t);
        long stop = System.currentTimeMillis();
        log("Total: "+tree.get());
        log("Total time: "+(stop-start)+"ms");
        try {
            FileWriter fw = new FileWriter("java-combining.txt", true);
            BufferedWriter bw = new BufferedWriter(fw);
            bw.write("java"+',');
            bw.write(Integer.toString(TH) +',');
            bw.write(Integer.toString(NUM) +',');
            bw.write(Long.toString(stop-start));
            bw.newLine();
            bw.close();
            System.out.println("Successfully wrote to the file.");
        } catch (IOException e) {
            System.out.println("An error occurred.");
            e.printStackTrace();
        }
        
        
    }

    static void log(String x) {
        System.out.println(x);
    }

    static long id() {
        return Thread.currentThread().getId();
    }
}

Overwriting TestCombiningTree.java


In [16]:
!javac TestCombiningTree.java

In [8]:
!java -ea TestCombiningTree 12 1000

6-leaves Combining tree.
Starting 12 threads doing increments ...
23: done in 24ms
16: done in 25ms
22: done in 25ms
17: done in 24ms
19: done in 37ms
25: done in 36ms
18: done in 38ms
24: done in 37ms
20: done in 39ms
14: done in 40ms
21: done in 39ms
15: done in 39ms
Total: 12000
Total time: 42ms
Successfully wrote to the file.


Counting Network

In [7]:
%%writefile Balancer.java
public class Balancer {
    boolean toggle = true;
    public synchronized int traverse() {
        try {
            if (toggle) {
            return 0;
            } else {
            return 1;
            }
        } finally {
            toggle = !toggle;
        }
    }
}

Overwriting Balancer.java


In [8]:
%%writefile Merger.java
public class Merger {
    Merger[] half; // two half-width merger networks
    Balancer[] layer; //final layer
    final int width;
    public Merger(int myWidth) {
        width = myWidth;
        layer = new Balancer[width / 2];
        for (int i = 0; i < width / 2; i++) {
            layer[i] = new Balancer();
        }
        if (width > 2) {
            half = new Merger[]{new Merger(width/2), new Merger(width/2)};
        }
    }
    public int traverse(int input) {
        int output = 0;
        
        if (width<=2) return layer[0].traverse();
        
        if (input < width / 2) {
            output = half[input % 2].traverse(input / 2);
        } else {
            output = half[1 - (input % 2)].traverse(input / 2);
        }
        return (2 * output) + layer[output].traverse();
    }
}

Overwriting Merger.java


In [9]:
%%writefile Bitonic.java
public class Bitonic {
    Bitonic[] half; // two half-width bitonic networks
    Merger merger; // final merger layer
    final int width; // network width
    public Bitonic(int myWidth) {
        width = myWidth;
        merger = new Merger(width);
        if (width > 2) {
            half = new Bitonic[]{new Bitonic(width/2), new Bitonic(width/2)};
        }
    }
    public int traverse(int input) {
        int output = 0;
        int subnet = input / (width / 2);
        if (width > 2) {
            output = half[subnet].traverse(input/2);
        }
        return merger.traverse((input>=width/2 ? (width/2) :0) + output);
    }
}

Overwriting Bitonic.java


In [29]:
%%writefile TestCounting.java

import java.util.*;
import java.util.concurrent.atomic.AtomicInteger;
import java.io.BufferedWriter;
import java.io.FileWriter;
import java.io.IOException;

public class TestCounting{
    //static int width = 4;
    //static int tokenCount = 1000;
    static int width;
    static int tokenCount;
    //static int[] counters = new int[width];
    //static Bitonic bitonic = new Bitonic(width);
    //static int[] tokens = new int[tokenCount];
    
    
    static Thread thread(int i, int[] tokens, int[] counters, Bitonic bitonic) {
        return new Thread(() -> {                          
            counters[bitonic.traverse(tokens[i])]+=1;
        });
    }
    
    public static void main(String[] args) {
        width = Integer.parseInt(args[0]);
        tokenCount = Integer.parseInt(args[1]);
        
        int[] tokens = new int[tokenCount];
        int[] counters = new int[width];
        Bitonic bitonic = new Bitonic(width);
        System.out.println(tokens.length);
        
        Random rand = new Random();
        for(int i=0; i<tokenCount; i++){
            tokens[i]=rand.nextInt(width);
        }
        
            
        Thread[] t = new Thread[tokenCount];
        long start = System.currentTimeMillis();
        for (int i=0; i<tokenCount; i++)
          t[i] = thread(i, tokens, counters, bitonic);
        for (int i=0; i<tokenCount; i++)
          t[i].start();
        try {
        for (int i=0; i<tokenCount; i++)
          t[i].join();
        } catch (InterruptedException e) {}
        long stop = System.currentTimeMillis();
               
        for (var i = 0; i < width; ++i)
        {
            System.out.println("Output: " + i +", Count: "+counters[i]);
        }
        
        System.out.println("Total time to traverse the network: " + (stop-start) +"ms");
        
        try {
            FileWriter fw = new FileWriter("java-counting.txt", true);
            BufferedWriter bw = new BufferedWriter(fw);
            bw.write("java"+',');
            bw.write(Integer.toString(width) +',');
            bw.write(Integer.toString(tokenCount) +',');
            bw.write(Long.toString(stop-start));
            bw.newLine();
            bw.close();
            System.out.println("Successfully wrote to the file.");
        } catch (IOException e) {
            System.out.println("An error occurred.");
            e.printStackTrace();
        }
        
    }
}

Overwriting TestCounting.java


In [30]:
!javac TestCounting.java

In [38]:
!java -ea TestCounting 4 10000

10000
Output: 0, Count: 2500
Output: 1, Count: 2500
Output: 2, Count: 2500
Output: 3, Count: 2500
Total time to traverse the network: 841ms
Successfully wrote to the file.
